In [1]:
import numpy as np
import pandas as pd
import sklearn

import cupy as cp
import cudf as df
import dask_cudf as gdf
import cuml as cm

In [3]:
import time

class Timer:
    def __init__(self, name=None):
        self.name = name

    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time.time()
        self.interval = self.end - self.start
        if self.name:
            print(f"[{self.name}] Elapsed time: {self.interval:.4f} seconds")
        else:
            print(f"Elapsed time: {self.interval:.4f} seconds")

In [8]:
with Timer("cpu"):
    arr = np.random.randint(10, size=(100000000, 2))
    df1 = pd.DataFrame(arr)

[cpu] Elapsed time: 2.8399 seconds


In [9]:
with Timer("gpu"):
    arr = cp.random.randint(10, size=(100000000, 2))
    df1 = df.DataFrame(arr)

[gpu] Elapsed time: 0.3999 seconds


In [10]:
x = cp.arange(6, dtype='f').reshape(2, 3)
y = cp.arange(3, dtype='f')

In [11]:
kernel = cp.ElementwiseKernel(
    'float32 x, float32 y', 'float32 z',
    '''
    if (x -2 > y) {
        z = x * y;
    } else {
        z = x + y;
    }
    ''', 'my_kernel'
)


In [12]:
kernel(x, y)

array([[ 0.,  2.,  4.],
       [ 0.,  4., 10.]], dtype=float32)

In [13]:
@cp.fuse(kernel_name='my_kernel')
def elementwise_mykernel(x, y):
    return cp.where(x-2 > y, x * y, x + y)

In [14]:
elementwise_mykernel(x, y)

array([[ 0.,  2.,  4.],
       [ 0.,  4., 10.]], dtype=float32)